Импортируем необходимые библиотеки

In [1]:
import pandas as pd

Загружаем данные из папки data

In [2]:
train_data = pd.read_csv('data/train-001.csv')
lectures_data = pd.read_csv('data/lectures.csv')
question_data = pd.read_csv('data/questions.csv')

Выводим первые 5 строк каждого датафрейма

In [3]:
display(train_data.head())
display(lectures_data.head())
display(question_data.head())

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


Выводим общую информацию о датафреймах 

In [4]:

lectures_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lecture_id  418 non-null    int64 
 1   tag         418 non-null    int64 
 2   part        418 non-null    int64 
 3   type_of     418 non-null    object
dtypes: int64(3), object(1)
memory usage: 13.2+ KB


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int64  
 3   content_id                      int64  
 4   content_type_id                 int64  
 5   task_container_id               int64  
 6   user_answer                     int64  
 7   answered_correctly              int64  
 8   prior_question_elapsed_time     float64
 9   prior_question_had_explanation  object 
dtypes: float64(1), int64(8), object(1)
memory usage: 7.5+ GB


In [6]:
question_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13523 non-null  int64 
 1   bundle_id       13523 non-null  int64 
 2   correct_answer  13523 non-null  int64 
 3   part            13523 non-null  int64 
 4   tags            13522 non-null  object
dtypes: int64(4), object(1)
memory usage: 528.4+ KB


Получим несколько интересных для исследования метрик

Процент правильно отвеченных вопросов:

In [7]:
correct_answers_count = (train_data['answered_correctly'] == 1).sum()
correct_answers_count / train_data.shape[0] * 100

64.45165763162764

Среднее время, затраченное на ответ

In [8]:
train_data['prior_question_elapsed_time'].mean()

25423.810042960275

Влияние объяснений на результат

In [9]:
# Разделение данных на две группы
group_with_explanation = train_data[train_data['prior_question_had_explanation'] == True]
group_without_explanation = train_data[train_data['prior_question_had_explanation'] == False]

# Подсчет процента правильных ответов в каждой группе
percentage_correct_with_explanation = (group_with_explanation['answered_correctly'] == 1).sum() / group_with_explanation.shape[0] * 100
percentage_correct_without_explanation = (group_without_explanation['answered_correctly'] == 1).sum() / group_without_explanation.shape[0] * 100
display(percentage_correct_with_explanation)
display(percentage_correct_without_explanation)

67.31885043701573

41.268734085072936